# Using the planner

Source : <https://github.com/microsoft/semantic-kernel/blob/main/dotnet/notebooks/05-using-the-planner.ipynb>

In [2]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 1.14.1"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.14.1-preview"

Installed Packages Microsoft.SemanticKernel, 1.14.1

Error: C:\Users\Administrator\.packagemanagement\nuget\Projects\2332--ff7d27be-270e-4eb7-b547-15d0f4041de5\Project.fsproj : error NU1103: Unable to find a stable package Microsoft.SemanticKernel.Planners.Handlebars with version (>= 0.0.0)
C:\Users\Administrator\.packagemanagement\nuget\Projects\2332--ff7d27be-270e-4eb7-b547-15d0f4041de5\Project.fsproj : error NU1103:   - Found 26 version(s) in nuget.org [ Nearest version: 1.14.1-preview ]
C:\Users\Administrator\.packagemanagement\nuget\Projects\2332--ff7d27be-270e-4eb7-b547-15d0f4041de5\Project.fsproj : error NU1103:   - Found 0 version(s) in C:\Program Files\dotnet\sdk\8.0.206\FSharp\library-packs